<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)

---

<center><h1> <font color="red">Introduction to Dask</font></h1></center>

## <font color="red">Reference Document</font>

- <a href="https://docs.dask.org/en/latest/why.html">Why Dask?</a>
- <a href="https://github.com/dask/dask-tutorial">dask-tutorial</a>
- <a href="https://www.manning.com/books/data-science-with-python-and-dask">Data Science with Python and Dask</a>
- <a href="https://www.manifold.ai/dask-and-machine-learning-preprocessing-tutorial">Dask and Machine Learning: Preprocessing Tutorial</a>
- <a href="https://www.youtube.com/watch?v=uGy5gT2vLdI&feature=youtu.be"> Working with the Python DASK library (video)</a>
- <a href="https://www.youtube.com/watch?v=t_GRK4L-bnw&feature=youtu.be">Who uses Dask (video)</a>

![fig_dask](https://miro.medium.com/max/1000/1*D6mSsdWECFLn6wJne4VTjg.png)


# <font color="red"> What is Dask?</font>

- A flexible library for parallel computing in Python that makes it easy to build intuitive workflows for ingesting and analyzing large, distributed datasets. 
- A native parallel analytics tool designed to integrate seamlessly with Numpy, Pandas, and Scikit-Learn. 
- An out-of-core (data is read into memory from disk on an as-needed basis) parallelization library that seamlessly integrates with existing NumPy and Pandas data structures to address the following:
     * **The vailable dataset does not fit in memory of a single machine.**
     * **The data processing task is time consuming and needs to be sped up.**
- Orchestrates parallel threads or processes for us and help speed up processing times.

Dask consists of several different components and APIs, which can be categorized into three layers: the scheduler, low-level APIs, and high-level APIs.

- Dask provides a few high-level constructs called Dask Bags, Dask DataFrames, and Dask Arrays. They provide an easy-to-use interface to parallelize many of the typical data transformations in ML workflows. 
- Dask allows the creation of highly customized job execution graphs by using their extensive Python API (e.g., `dask.delayed`) and integration with existing data structures.


![fig_layers](http://bicortex.com/bicortex/wp-content/post_content//2019/06/Dask_APIs_Architecture.png)
Image Source: bicortex.com


The diagram below describes the steps Dask takes to manipulate data.

- The operation is broken down into a sequence of operations on smaller partitions of our data (without having to read the whole dataset into memory).
- Dask reads each partition as it is needed and computes the intermediate results. 
- The intermediate results are aggregated into the final result.
- Dask handles all of that sequencing internally for us. 
- On a single machine, Dask can use threads or processors to parallelize these operations. 

![fig_proc](https://www.manifold.ai/hs-fs/hubfs/Blog%20Post%20Illos/ML%20pipelines%20-%20dask%20single%20machine.jpeg?width=600&name=ML%20pipelines%20-%20dask%20single%20machine.jpeg)
Image Source: www.manifold.ai


**Advantages of Using Dask**

- Fully implemented in Python and natively scales NumPy, Pandas, and scikit-learn.
- Can be used effectively to work with both medium datasets on a single machine and large datasets on a cluster.
- Can be used as a general framework for parallelizing most Python objects.
- Has a very low configuration and maintenance overhead.



>Dask provides high-level Array, Bag, and DataFrame collections that mimic NumPy, lists, and Pandas but can operate in parallel on datasets that don’t fit into main memory. Dask’s high-level collections are alternatives to NumPy and Pandas for large datasets.

In [ ]:
#!python -m pip install dask[dataframe] --upgrade

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import dask
import dask.array as da
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 

In [ ]:
print("Pandas version: ", pd.__version__)
print("Dask   version: ", dask.__version__)

# <font color="red"> Parallelize Code with `dask.delayed`</font>

- A simple way to parallelize the code.
- Allows users to delay function calls into a task graph with dependencies.
- Systems like `dask.dataframe` are built with `dask.delayed`.

**Simple Example**

Consider the following functions:

In [ ]:
import time

def increment(x):
    time.sleep(1.0)
    return x + 1

def double(x):
    time.sleep(1.0)
    return 2 * x

def add(x, y):
    time.sleep(1.0)
    return x + y

In [ ]:
%%time

x = increment(1)
y = increment(2)
z = add(x, y)

- We use the `dask.delayed` decorator to parallelize the functions `increment` and `add`.
- By decorating the functions, we record what we want to compute as tasks into graphs that will be run later on parallel hardware.

In [ ]:
xd = dask.delayed(increment)(1)
yd = dask.delayed(increment)(2)
zd = dask.delayed(add)(xd, yd)
zd

- When we call the delayed version by passing the arguments, exactly as before, but the original function isn't actually called yet.
- A delayed object is made, which keeps track of the function to call and the arguments to pass to it.
- We use the `visualize` method (relies on the `graphviz` package) that provide a visual representation of the operations being performed.

In [ ]:
zd.visualize(rankdir='LR')

- Note that we have not physically calculated **total** yet.
- We need to apply the `compute` method to get the answer. 
- <font color="red">It is only here that the data are loaded into memory for calculations</font>.

In [ ]:
%%time
dask.compute(zd)

**Using `delayed` in Loops**

Consider the sequential code with two for-loops:

In [ ]:
%%time

n = 10
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = increment(x)
    z = double(y)
    out.append(z)
    
total = 0
for z in out:
    total = add(total, z)

total

We can parallelize the above using the `delayed` decorator:

In [ ]:
%%time

n = 10
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = dask.delayed(increment)(x)
    z = dask.delayed(double)(y)
    out.append(z)
    
totald = 0
for z in out:
    totald = dask.delayed(add)(totald, z)

totald

We can also get the visual representation through a task graph.

In [ ]:
totald.visualize()

In [ ]:
%%time
dask.compute(totald)

### Exercise

Use the `delayed` decorator to parallelize the code below:

In [ ]:
def is_odd(x):
    return (x%2)==1

In [ ]:
%%time

n = 10
data = [i+1 for i in range(n)]

results = list()

for x in data:
    if is_odd(x):
        y = double(x)
    else:
        y = increment(x)
    results.append(y)

total = sum(results)
print(total)

### Example: Palindromic Words

- A palindromic word is a word which characters read the same backward as forward. 
- Some examples of palindromes are `redivider`, `deified`, `civic`, `radar`, `level`, `rotor`, `kayak`, `reviver`, `racecar`, `madam`, and `refer`.

We want to find the number of palindromes from a list of words.

In [ ]:
import time
def is_palindrome(s):
    return s.upper() == s.upper()[::-1]

In [ ]:
string_list = ['redivider', 'deified', 'civic', 'radar', 'level',
               'rotor', 'kayak', 'reviver', 'racecar', 'madam', 'refer',
               'Being',  'man', 'not', 'without', 'frequent', 'consciousness',
               'that', 'there', 'was', 'some', 'charm', 'this', 'life', 'stood',
               'still', 'after', 'looking', 'sky', 'useful', 'instrument',
               'regarded', 'appreciative', 'spirit', 'work', 'art',
               'superlatively', 'beautiful', 'moment', 'seemed',
               'impressed', 'with', 'speaking', 'loneliness', 'scene',
               'rather', 'complete', 'abstraction', 'from', 'compass',
               'sights', 'sounds', 'man', 'Human', 'shapes', 'interferences',
               'troubles', 'joys', 'were', 'they', 'were', 'there',
               'seemed', 'shaded', 'hemisphere', 'globe', 'sentient', 'being',
               'save', 'himself']

**Using Regular Python**

In [ ]:
%%time
palindromes_py = [is_palindrome(s) for s in string_list]
total_py = sum(palindromes_py)

**Using Dask**

In [ ]:
palindromes_da = [dask.delayed(is_palindrome)(s) for s in string_list]
total_da = dask.delayed(sum)(palindromes_da)

In [ ]:
total_da.visualize()

In [ ]:
%%time
result = total_da.compute()

**<font color="red">Important Lesson</font>**

- The `delayed` decorator adds overhead.
- It is good not to use it when a task requires a little amount of time.

# <font color="red"> Dask Array</font>

- Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid. 
    - _Parallel_: Uses all of the cores on your computer
    - _Larger-than-memory_: Lets you work on datasets that are larger than your available memory by breaking up your array into many small pieces, operating on those pieces in an order that minimizes the memory footprint of your computation, and effectively streaming data from disk.
    - _Blocked Algorithms_: Perform large computations by performing many smaller computations
- They support a large subset of the Numpy API.

![fig_array](https://miro.medium.com/max/1388/1*JfQnXJ5_R104bPyE8_XhwQ.png)

**Create a Dask Array**

- Create a 20000x20000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). 
- There are 100 (10x10) numpy arrays of size 1000x1000.

In [ ]:
x = da.random.random((20000, 20000), chunks=(1000, 1000))
x

We can use Numpy syntax:

In [ ]:
y = x + x.T
y.shape

In [ ]:
z = y[::2, 5000:].mean(axis=1)
z

In [ ]:
z.visualize(rankdir="LR")

Use the **`compute()`** function if you want your result as a NumPy array.

In [ ]:
w = z.compute()
print(type(w), w.shape )

**Persit Data in Memory**

- If you have the available RAM for your dataset then you can persist data in memory.
- This allows future computations to be much faster.

In [ ]:
%time y.sum().compute()

In [ ]:
y = y.persist()

In [ ]:
%time y[0, 0].compute()

In [ ]:
%time y.sum().compute()

**Numpy against Dask**

In [ ]:
%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
#y

In [ ]:
%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
x = x.persist()
y = x.mean(axis=0)[::100] 
#y.compute() 

# <font color="red"> Dask DataFrames</font>

- Pandas is great for tabular datasets that fit in memory. 
- Dask becomes useful when the dataset you want to analyze is larger than your machine's RAM. 
- Dask DataFrames:
     - Coordinate many Pandas DataFrames, partitioned along an index. 
     - Support a large subset of the Pandas API.
- One operation on a Dask DataFrame triggers many Pandas operations on the constituent pandas DataFrames in a way that is mindful of potential parallelism and memory constraints.
- Some of the operations that are really fast if you use Dask Dataframes:
     - Arithmetic operations (multiplying or adding to a Series)
     - Common aggregations (`mean`, `min`, `max`, `sum`, etc.)
     - Calling `apply`
     - Calling `value_counts()`, `drop_duplicates()` or `corr()`
     - Filtering with `loc`, `isin`, and row-wise selection

![fig_df](https://pythondata.com/wp-content/uploads/2016/11/Screen-Shot-2016-11-24-at-6.52.24-PM-168x300.png)

### <font color="green"> NYC Flights Dataset</font>

Data is specific to flights out of the three airports in the New York City area.

Download the remote data:

In [ ]:
import urllib.request

print("\t Downloading NYC dataset...", end="\n", flush=True)

url = "https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz"
filename, header = urllib.request.urlretrieve(url, "nycflights.tar.gz")

print("\t Done!", flush=True)

Extract the `.csv` files from the tar file:

In [ ]:
import tarfile

with tarfile.open(filename, mode="r:gz") as flights:
     flights.extractall("data/")

Read all the files at once:

In [ ]:
df = dd.read_csv(os.path.join("data", "nycflights", "*.csv"), 
                parse_dates={"Date": [0, 1, 2]})
df

- The representation of the dataframe object contains no data. 
- `pandas.read_csv` reads in the entire file before inferring datatypes.
- `dask.dataframe.read_csv` only reads in a sample from the beginning of the file (or first file). These inferred datatypes are then enforced when reading all partitions.

We can display the first few rows:

In [ ]:
df.head()

If we display the last few rows, we have a problem:

In [ ]:
df.tail()

- There is an issue with the data types of few columns.
- The datatypes inferred in the sample are incorrect.
- We can fix it by reading the files again and specify the appropriate data types.

In [ ]:
df = dd.read_csv(os.path.join("data", "nycflights", "*.csv"), 
                parse_dates={"Date": [0, 1, 2]},
                dtype={'TailNum': str,
                       'CRSElapsedTime': float,
                       'Cancelled': bool})

In [ ]:
df.tail()

### <font color="blue">Perform Operations as with `Pandas DataFrames`</font>

**Maximum value of a column**:

- We now want to compute the maximum of the `DepDelay` column.
- With `Pandas`, we would loop over each file to find the individual maximums, then find the final maximum over all the individual maximums.
- `dask.dataframe` allows us to write pandas-like code that operates on large than memory datasets in parallel.

In [ ]:
%time df.DepDelay.max().compute()

Using `Pandas`, we will have something like:

```python
   maxes = []
   for fn in filenames:
       df = pd.read_csv(fn)
       maxes.append(df.DepDelay.max())

   final_max = max(maxes)
```

**Plotting**

In [ ]:
df[df.Dest == 'PIT'].compute().plot(kind='scatter', 
                                    x="DayOfWeek", 
                                    y="DepDelay")

**Other Operations**

Number of non-cancelled flights:

In [ ]:
len(df[~df.Cancelled])

Number of non-cancelled flights were taken from each airport:

In [ ]:
df[~df.Cancelled].groupby('Origin').Origin.count().compute()

Average departure delay from each airport:

In [ ]:
df.groupby("DayOfWeek").DepDelay.mean().compute()

Groub by destinations and count:

In [ ]:
df.groupby("Dest").count().compute()

**Sharing Intermediate Results**

- We sometimes do the same operation more than once. 
- For most operations, `dask.dataframe` hashes the arguments, allowing duplicate computations to be shared, and only computed once.

In [ ]:
non_cancelled = df[~df.Cancelled]
mean_delay = non_cancelled.DepDelay.mean()
std_delay = non_cancelled.DepDelay.std()

In [ ]:
%%time
mean_delay_res = mean_delay.compute()
std_delay_res = std_delay.compute()

We pass both to a single `compute` call:

In [ ]:
%%time

mean_delay_res, std_delay_res = da.compute(mean_delay, std_delay)

The task graphs for both results are merged when calling dask.compute, allowing shared operations to only be done once instead of twice.

In [ ]:
dask.visualize(mean_delay, std_delay)

### <font color="blue">Example of Machine Learning with Dask</font>

Grab columns from the Dask DataFrame:

In [ ]:
df_train = df[["CRSDepTime", "CRSArrTime", "Cancelled"]]
df_train

You can query the shape (note delayed # of sample):

In [ ]:
df_train.shape

In [ ]:
len(df_train)

**Basic EDA**

We can get descriptive statistics:

In [ ]:
df_train.describe().compute()

Perform searches and operations on the data:

In [ ]:
df_train.isnull().sum().compute()

**Create the Model**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.Dense(20, 
                       input_dim=len(df_train), 
                       activation='relu'))
model.add(layers.Dense(1,  activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="sgd", )

**Train the Model**

Generate batches of data:

In [ ]:
def batch_data_generator(df, fraction=0.01):
    while True:
          batch = df.sample(frac=fraction)
          X = batch.iloc[:, :-1]
          y = batch.iloc[:, -1]
          yield (X.compute(), y.compute())

We never run of memory while doing the training:

```
   steps_per_epoch * batch_size = number_of_rows_in_train_data
```

In [ ]:
model.fit_generator(generator=batch_data_generator(df_train),
                    steps_per_epoch=100)

# <font color="red"> Schedulers</font>

- After Dask generates the task graphs, it needs to execute them on parallel hardware. 
- It is the role of a task scheduler. 
- There are different task schedulers. Each will consume a task graph and compute the same result, but with different performance characteristics.

![Schedulers](https://docs.dask.org/en/latest/_images/collections-schedulers.png)
Image Source: https://docs.dask.org/

To execute the task graphs there are two types of schedulers:
* **Single machine**: Provides basic features on a local process or thread pool. It is simple and cheap to use, although it can only be used on a single machine and does not scale
* **Distributed**: Offers more features, but also requires a bit more effort to set up. It can run locally or distributed across a cluster.

In [ ]:
%%time

n = 10
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = dask.delayed(increment)(x)
    z = dask.delayed(double)(y)
    out.append(z)
    
totald = 0
for z in out:
    totald = dask.delayed(add)(totald, z)

totald

**Single thread**

- The single-threaded synchronous scheduler executes all computations in the local thread with no parallelism at all.

In [ ]:
%time totald.compute(scheduler='synchronous')

**Local threads**

Uses `multiprocessing.pool.ThreadPool`

Use all the processors

In [ ]:
%time totald.compute(scheduler='threads')

Use some of the processors:

In [ ]:
%time totald.compute(scheduler='threads', num_workers=2)

**Local processes**

- The multiprocessing scheduler executes computations with a local `multiprocessing.Pool`.
- Every task and all of its dependencies are shipped to a local process, executed, and then their result is shipped back to the main process. 
- Moving data to remote processes and back can introduce performance penalties, particularly when the data being transferred between processes is large. 
- The multiprocessing scheduler is an excellent choice when workflows are relatively linear, and so does not involve significant inter-task data transfer as well as when inputs and outputs are both small, like filenames and counts.

Use all the processors:

In [ ]:
import multiprocessing
print (multiprocessing.cpu_count())

In [ ]:
%time result = totald.compute(scheduler='processes')

Use some of the processors:

In [ ]:
%time result = totald.compute(scheduler='processes', num_workers=2)

**Dask distributed scheduler: Run through a Dashboard**

- The Dask distributed scheduler can either be setup on a cluster or run locally on a personal machine. 
- It is a centrally managed, distributed, dynamic task scheduler. The central dask-scheduler process coordinates the actions of several dask-worker processes spread across multiple machines and the concurrent requests of several clients.
- The scheduler is asynchronous and event-driven, simultaneously responding to requests for computation from multiple clients and tracking the progress of multiple workers.
- The event-driven and asynchronous nature makes it flexible to concurrently handle a variety of workloads coming from multiple users at the same time while also handling a fluid worker population with failures and additions. 
- Workers communicate amongst each other for bulk data transfer over TCP.
- To set up `dask.distributed`, we need to create client instance by calling `Client` class from `dask.distributed`. 
- It will internally create a dask scheduler and dask workers. 
- We will get the link of the dashboard where we can analyze tasks running in parallel. 
- We can pass a number of workers (using the `n_workers` argument) and threads to use per worker process (using the `threads_per_worker` argument).
- As soon as you create a client, Dask will automatically start using it.

In [ ]:
from dask.distributed import Client
client = Client(n_workers=2, threads_per_worker=4)
client

In [ ]:
import random

def random_slow_add(x, y):
    time.sleep(random.randrange(3,10))
    return x + y

In [ ]:
results = []

for x in data:
    y = dask.delayed(random_slow_add)(x, 1)
    results.append(y)
    
total = dask.delayed(sum)(results)

In [ ]:
%time result = total.compute()
result

Shut down the cluster:

In [ ]:
client.close()

**<font color="red">Things to Consider</font>**

- Each Dask task has overhead (about 1 ms). If you have a lot tasks this overhead can add up. It is a good idea to give each task more than a few seconds of work.
- To better understand how your program is performing, check the [Dask Performance Diagnostics](https://distributed.dask.org/en/latest/diagnosing-performance.html) documentation. You can also view the [video](https://docs.dask.org/en/stable/diagnostics-distributed.html) to find out how to group your work into fewer, more substantial tasks. This might mean that you call lazy operations at once instead of individually. This might also repartitioning your dataframe(s).
- A good rule of thumb for choosing number of threads per Dask worker is to choose the square root of the number of cores per processor. In general more threads per worker are good for a program that spends most of its time in NumPy, SciPy, Numba, etc., and fewer threads per worker are better for simpler programs that spend most of their time in the Python interpreter.
- The Dask scheduler runs on a single thread, so assigning it its own node is a waste.
- There is no hard limit on Dask scaling. The task overhead though will eventually start to swamp your calculation depending on how long each task takes to compute. 

## <font color="blue"> Example with DataFrame</font>

Build a Pandas DataFrame with 100000 rows and two columns with values selected randomly between 1 and 1000.

In [ ]:
num_rows = 100000
df = pd.DataFrame({'X':np.random.randint(1000, size=num_rows),
                   'Y':np.random.randint(1000, size=num_rows)})
df

Write a function that computes the sum of square for each column of the DataFrame.

In [ ]:
def add_squares(df):
    return df.X**2 + df.Y**2

Measure the time it takes to call the function:

In [ ]:
%%timeit
df['add_squares'] = df.apply(add_squares,axis=1)

In [ ]:
df

### <font color="green">Parallelize using Dask `Map_Partition`</font>

We construct a Dask DataFrame from pandas dataframe using `from_pandas` function and specify the number of partitions (`nparitions`) to break this dataframe into.

```python
   dd = ddf.from_pandas(df, npartitions=N)
```

`ddf` is the name you imported Dask Dataframes with, and `npartitions` is an argument telling the Dataframe how you want to partition it.

Each partition will run on a different thread, and communication between them will become too costly if there are too many.

We will break into 4 partitions (number of available cores):

In [ ]:
ddf = dd.from_pandas(df, npartitions=4)

We will apply `add_squares` method on each of these partitions:

In [ ]:
%%timeit
ddf['z'] = ddf.map_partitions(add_squares, 
                               meta=(None, 'int64')).compute()

In [ ]:
def myfunc(x, y):
    return y * (x**2 + 1)

In [ ]:
%%timeit
df1 = df.apply(lambda row: myfunc(row.X, row.Y), axis=1)

In [ ]:
import multiprocessing
ddf = dd.from_pandas(df, npartitions=4*multiprocessing.cpu_count())
ddf

In [ ]:
%%timeit
ddfz = ddf.map_partitions(lambda data: 
                              data.apply(lambda row: myfunc(row.X, row.Y), axis=1)).compute(scheduler='processes')

# <font color="red">Dask and GPUs</font>

- Dask can also help to scale out large array and dataframe computations by combining the Dask Array and DataFrame collections with a GPU-accelerated array or dataframe library.
- The RAPIDS libraries provide a GPU accelerated Pandas-like library, `cuDF`, which interoperates well and is tested against Dask DataFrame.
- You can convert a Pandas-backed Dask DataFrame to a cuDF-backed Dask DataFrame.

![rapids](https://pbs.twimg.com/media/D2CeyaYVAAAe3kM.jpg)
Image Source: NVIDIA

- Graphics Processing Units (GPUs) are custom designed to be very efficient at handling computer graphics and image processing.
- Central Processing Units (CPUs) handle computations serially, meaning the logic in handled in one stream: the next task will complete when the subsequent task has finished. CPUs can execute tasks in parallel across cores. For example, most computer CPUs tend to have either two, four or six cores.
- In comparison, GPUs have hundreds of 'cores'. This massively parallel architecture is what gives the GPU its high compute performance.

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [ ]:
!nvidia-smi

In [ ]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
    raise Exception("""
       Unfortunately this instance does not have a T4 GPU.
       Please make sure you've configured Colab to request a GPU instance type. 
       Sometimes Colab allocates a Tesla K80 instead of a T4. 
       Resetting the instance.
       If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
    print('Great! You got the right kind of GPU!')

#Setup:

1. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
2. Install RAPIDS libraries
3. Set necessary environment variables
4. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions

In [ ]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
    shutil.copy('/usr/local/lib/'+fn, os.getcwd())

In [ ]:
!pip install dask

In [ ]:
!pip install dask_cuda

In [ ]:
from dask.distributed import Client, LocalCluster, wait, progress


# we have one GPU, so limit Dask's workers and threads to exactly 1
cluster = LocalCluster(processes=False)
client = Client(cluster)
client

In [ ]:
!pip install dask_xgboost
!pip install dask_cudf
!pip install dask_cuml

In [ ]:
import dask
import dask_cudf
import dask_xgboost

print('Dask Version:', dask.__version__)
print('Dask cuDF Version:', dask_cudf.__version__)
print('Dask XGBoost Version:', dask_xgboost.__version__)

In [ ]:
import cudf, cuml, xgboost
import dask_cudf, dask_cuml, dask_xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [ ]:
# Creating a linearly separable dataset using Gaussian Distributions.
# The first half of the number in Y is 0 and the other half 1.
# Therefore I made the first half of the two features quite different from
# the second half of the features (setting the value of the means quite 
# similar) so that make quite simple the classification between the 
# classes (the data is linearly separable).
dataset_len = 200
dlen = int(dataset_len/2)
X_11 = pd.Series(np.random.normal(2,2,dlen))
X_12 = pd.Series(np.random.normal(9,2,dlen))
X_1 = pd.concat([X_11, X_12]).reset_index(drop=True)
X_21 = pd.Series(np.random.normal(1,3,dlen))
X_22 = pd.Series(np.random.normal(7,3,dlen))
X_2 = pd.concat([X_21, X_22]).reset_index(drop=True)
X_31 = pd.Series(np.random.normal(3,1,dlen))
X_32 = pd.Series(np.random.normal(3,4,dlen))
X_3 = pd.concat([X_31, X_32]).reset_index(drop=True)
Y = pd.Series(np.repeat([0,1],dlen))
df = pd.concat([X_1, X_2, X_3, Y], axis=1)
df.columns = ['X1', 'X2', 'X3', 'Y']
df.head()

In [ ]:
X = df.drop(['Y'], axis = 1) #.values
y = df['Y']

# cuDF and cuML Examples #

Now you can run code! 

What follows are basic examples where all processing takes place on the GPU.

#[cuDF](https://github.com/rapidsai/cudf)#

Load a dataset into a GPU memory resident DataFrame and perform a basic calculation.

Everything from CSV parsing to calculating tip percentage and computing a grouped average is done on the GPU.

_Note_: You must import nvstrings and nvcategory before cudf, else you'll get errors.

In [ ]:
import nvstrings, nvcategory, cudf
import io, requests

# download CSV file from GitHub
url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

# read CSV from memory
tips_df = cudf.read_csv(io.StringIO(content))
tips_df['tip_percentage'] = tips_df['tip']/tips_df['total_bill']*100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

#[cuML](https://github.com/rapidsai/cuml)#

This snippet loads a 

As above, all calculations are performed on the GPU.

In [ ]:
import cuml

# Create and populate a GPU DataFrame
df_float = cudf.DataFrame()
df_float['0'] = [1.0, 2.0, 5.0]
df_float['1'] = [4.0, 2.0, 1.0]
df_float['2'] = [4.0, 2.0, 1.0]

# Setup and fit clusters
dbscan_float = cuml.DBSCAN(eps=1.0, min_samples=1)
dbscan_float.fit(df_float)

print(dbscan_float.labels_)